# Add/import required packages

In [ ]:
using Pkg
Pkg.add(url="https://github.com/giordamaug/WiSARDj")
Pkg.add("MLJ")
Pkg.add("MLJBase")
Pkg.add("DataFrames")
Pkg.add("MLBase")
Pkg.add("RDatasets")
import RDatasets
using MLJ

    Updating git-repo `https://github.com/giordamaug/WiSARDj`


# Load the "iris" dataset

In [ ]:
iris = RDatasets.dataset("datasets", "iris");
first(iris, 3) |> pretty

# Classification wit WiSARDj

In [ ]:
using WiSARDj.MLJInterface: WiSARDClassifier
using MLJBase, MLBase, DataFrames

X = Matrix(DataFrames.select(iris, Not([:Species])))
y = vec(Matrix(DataFrames.select(iris, [:Species])))

model = WiSARDClassifier()
train, test = MLJ.partition(eachindex(y), 0.8, shuffle=true, rng=1)
MLJ.fit(model, Any, X[train,:], y[train,:])
ŷ = MLJ.predict(model, Any, X[test,:])
y_targets = y[test,:]
accuracy = sum(ŷ .== y_targets) / length(y_targets)
println("accuracy: $accuracy")
MLJBase.ConfusionMatrix()(ŷ, coerce(vec(y_targets), OrderedFactor))  # I don't know why we need to coerce! 

In [ ]:
features, targets = MLJBase.@load_boston
features = DataFrames.DataFrame(features)
@show size(features)
@show targets[1:3]
first(features, 3) |> pretty